In [1]:
import pandas as pd
df = pd.read_csv("salaries.csv")
df.shape

(16, 4)

In [2]:
df.head()

,company,job,degree,salary_more_then_100k
0,google,sales executive,bachelors,0
1,google,sales executive,masters,0
2,google,business manager,bachelors,1
3,google,business manager,masters,1
4,google,computer programmer,bachelors,0


In [3]:
inputs = df.drop(columns = 'salary_more_then_100k')
target = df.salary_more_then_100k

In [4]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn import tree

In [5]:
le_com = LabelEncoder()
le_job = LabelEncoder()
le_degree = LabelEncoder()

In [6]:
inputs.company = le_com.fit_transform(inputs.company)
inputs.job = le_job.fit_transform(inputs.job)
inputs.degree = le_degree.fit_transform(inputs.degree)

In [7]:
def encode(df,column):
    ohe = ColumnTransformer(
        transformers = [(column , OneHotEncoder(), [0])] ,remainder = 'passthrough'
    )
    x = ohe.fit_transform(df)
    x = x[:,1:] # drop column prevent dummy trap
    df.drop(columns = df.columns[0],inplace=True)
    return x

In [8]:
company = encode(inputs,inputs.columns[0])
job = encode(inputs,inputs.columns[0])
degree = encode(inputs,inputs.columns[0])

In [9]:
import numpy as np
final = np.concatenate((company,job,degree),axis = 1)

In [10]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(final,target,test_size=0.1)

In [11]:
from sklearn.model_selection import GridSearchCV
dtree = GridSearchCV(tree.DecisionTreeClassifier(), {
    'criterion' : ['gini','entropy'],
    'splitter': ['best','random'],
    'max_depth' : [1,2,3,4,5]
}, cv=3, return_train_score=True)
dtree.fit(x_train,y_train)

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [1, 2, 3, 4, 5],
                         'splitter': ['best', 'random']},
             return_train_score=True)

In [12]:
pd.DataFrame(dtree.cv_results_)[['param_criterion','param_splitter','param_max_depth','mean_test_score']]

,param_criterion,param_splitter,param_max_depth,mean_test_score
0,gini,best,1,0.500000
1,gini,random,1,0.700000
2,gini,best,2,0.500000
3,gini,random,2,0.633333
4,gini,best,3,0.500000
5,gini,random,3,0.700000
6,gini,best,4,0.650000
7,gini,random,4,0.766667
8,gini,best,5,0.516667
9,gini,random,5,0.716667


In [13]:
dtree.score(x_test,y_test)

1.0

In [14]:
#simple testing
pred = dtree.predict(x_test)
pred == y_test

13    True
4     True
Name: salary_more_then_100k, dtype: bool